In [1]:
import pandas as pd
import difflib

def string_diff(str1, str2):
    seq_matcher = difflib.SequenceMatcher(None, str1, str2)
    return seq_matcher.ratio() != 1

def compare_files(file1, file2, keys):
    diff_dfs = []

    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    df1.set_index(keys, inplace=True)
    df2.set_index(keys, inplace=True)

    diff_df = pd.DataFrame(index=df1.index)

    for column in df1.columns:
        for i in df1.index:
            if pd.isna(df1.at[i, column]) or pd.isna(df2.at[i, column]):
                diff_df.at[i, column + "_diff"] = df1.at[i, column] != df2.at[i, column]
                if df1.at[i, column] != df2.at[i, column]:
                    diff_df.at[i, column + "_value1"] = df1.at[i, column]
                    diff_df.at[i, column + "_value2"] = df2.at[i, column]
            else:
                diff_df.at[i, column + "_diff"] = string_diff(str(df1.at[i, column]), str(df2.at[i, column]))
                if string_diff(str(df1.at[i, column]), str(df2.at[i, column])):
                    diff_df.at[i, column + "_value1"] = df1.at[i, column]
                    diff_df.at[i, column + "_value2"] = df2.at[i, column]

    diff_dfs.append(diff_df)

    return pd.concat(diff_dfs)


In [2]:

# specify files

file1 = 'version1.csv'
file2 = 'version2.csv'
keys = ['key_column1', 'key_column2']  # adjust this to your key columns

diff_df = compare_files(file1, file2, keys)
diff_df


,,data_column1_diff,data_column1_value1,data_column1_value2,data_column2_diff,data_column2_value1,data_column2_value2
key_column1,key_column2,,,,,,
key1,subkey1,False,NaN,NaN,False,NaN,NaN
key2,subkey2,True,World,Earth,False,NaN,NaN
key3,subkey3,True,Example,Sample,True,789.0,100.0


In [3]:
def summarize_diff(diff_df):
    summary_data = []

    for row in diff_df.itertuples():
        for column in diff_df.columns:
            if "_diff" in column and row._asdict()[column] and row._asdict()[column.replace('_diff', '_value1')] != row._asdict()[column.replace('_diff', '_value2')]:
                keys_dict = dict(zip(keys, row.Index))
                keys_dict.update({
                    'column': column.replace('_diff', ''),
                    'value_file1': row._asdict()[column.replace('_diff', '_value1')],
                    'value_file2': row._asdict()[column.replace('_diff', '_value2')],
                })
                summary_data.append(keys_dict)

    summary_df = pd.DataFrame(summary_data)
    return summary_df

summary_df = summarize_diff(diff_df)
print(summary_df)


  key_column1 key_column2        column value_file1 value_file2
0        key2     subkey2  data_column1       World       Earth
1        key3     subkey3  data_column1     Example      Sample
2        key3     subkey3  data_column2       789.0       100.0


In [4]:
summary_df

,key_column1,key_column2,column,value_file1,value_file2
0,key2,subkey2,data_column1,World,Earth
1,key3,subkey3,data_column1,Example,Sample
2,key3,subkey3,data_column2,789.0,100.0


In [13]:
import pandas as pd
import difflib
import numpy as np
import timeit

def string_diff(str1, str2):
    seq_matcher = difflib.SequenceMatcher(None, str1, str2)
    return seq_matcher.ratio() != 1

def compare_files(base, target, keys):
    start = timeit.default_timer()

    chunksize = 10 ** 6  # Define your chunksize here according to your system's memory
    summary_data = []

    for chunk1, chunk2 in zip(pd.read_csv(base, chunksize=chunksize), pd.read_csv(target, chunksize=chunksize)):
        chunk1.set_index(keys, inplace=True)
        chunk2.set_index(keys, inplace=True)

        # Combine columns from both chunks
        columns = set(chunk1.columns).union(set(chunk2.columns))

        for column in columns:
            for row_index in chunk1.index.intersection(chunk2.index):
                value1 = chunk1.at[row_index, column] if column in chunk1.columns and row_index in chunk1.index else np.nan
                value2 = chunk2.at[row_index, column] if column in chunk2.columns and row_index in chunk2.index else np.nan

                description = ''

                if pd.isna(value1) and pd.isna(value2):
                    continue  # Skip if both are NaN
                
                if pd.isna(value1) and not pd.isna(value2):
                    description = 'Value is missing in base'
                elif not pd.isna(value1) and pd.isna(value2):
                    description = 'Value is missing in target'
                elif string_diff(str(value1), str(value2)):
                    description = 'Values do not match'

                if description:
                    keys_dict = dict(zip(keys, row_index))
                    keys_dict.update({
                        'column': column,
                        'value_base': value1,
                        'value_target': value2,
                        'description': description
                    })
                    summary_data.append(keys_dict)

    summary_df = pd.DataFrame(summary_data)

    end = timeit.default_timer()
    print('Time: ', end - start, 'seconds')
    
    return summary_df

# specify files
base = 'version1.csv'
target = 'version2.csv'
keys = ['key_column1', 'key_column2']  # adjust this to your key columns

summary_df = compare_files(base, target, keys)
summary_df


Time:  0.02402650000021822 seconds


,key_column1,key_column2,column,value_base,value_target,description
0,key2,subkey2,data_column1,World,Earth,Values do not match
1,key3,subkey3,data_column1,Example,Sample,Values do not match
2,key1,subkey1,data_column3,NaN,A,Value is missing in base
3,key3,subkey3,data_column3,NaN,C,Value is missing in base
4,key3,subkey3,data_column2,789,100,Values do not match


In [24]:
import os
import pandas as pd
import difflib
import numpy as np
import timeit

def string_diff(str1, str2):
    seq_matcher = difflib.SequenceMatcher(None, str1, str2)
    return seq_matcher.ratio() != 1

def compare_files(base, target, keys):
    chunksize = 10 ** 6  # Define your chunksize here according to your system's memory
    summary_data = []

    for chunk1, chunk2 in zip(pd.read_csv(base, chunksize=chunksize), pd.read_csv(target, chunksize=chunksize)):
        chunk1.set_index(keys, inplace=True)
        chunk2.set_index(keys, inplace=True)

        # Combine columns from both chunks
        columns = set(chunk1.columns).union(set(chunk2.columns))

        for column in columns:
            for row_index in chunk1.index.intersection(chunk2.index):
                value1 = chunk1.at[row_index, column] if column in chunk1.columns and row_index in chunk1.index else np.nan
                value2 = chunk2.at[row_index, column] if column in chunk2.columns and row_index in chunk2.index else np.nan

                description = ''

                if pd.isna(value1) and pd.isna(value2):
                    continue  # Skip if both are NaN
                
                if pd.isna(value1) and not pd.isna(value2):
                    description = 'Value is missing in base'
                elif not pd.isna(value1) and pd.isna(value2):
                    description = 'Value is missing in target'
                elif string_diff(str(value1), str(value2)):
                    description = 'Values do not match'

                if description:
                    keys_dict = dict(zip(keys, row_index))
                    keys_dict.update({
                        'column': column,
                        'value_base': value1,
                        'value_target': value2,
                        'description': description
                    })
                    summary_data.append(keys_dict)

    summary_df = pd.DataFrame(summary_data)
    return summary_df

base_dir = 'base'
target_dir = 'target'
keys = ['key_column1', 'key_column2']  # adjust key columns

for filename in os.listdir(base_dir):
    base_file = os.path.join(base_dir, filename)
    target_file = os.path.join(target_dir, filename)

    if os.path.isfile(target_file):
        start = timeit.default_timer()

        summary_df = compare_files(base_file, target_file, keys)

        # Add additional columns to the dataframe
        summary_df['filename'] = filename
        summary_df['business_unit'] = 'TH'  
        summary_df['comparison_type'] = 'LBU_Submission_to_Tool_Pack' 

        end = timeit.default_timer()
        print('Time: ', end - start, 'seconds')

#         print(summary_df)

        #In progress: Handle files that is not in the other folder

Time:  0.0081583999999566 seconds
  key_column1 key_column2        column value_base value_target  \
0        key2     subkey2  data_column1      World        Earth   
1        key3     subkey3  data_column1    Example       Sample   
2        key1     subkey1  data_column3        NaN            A   
3        key3     subkey3  data_column3        NaN            C   
4        key3     subkey3  data_column2        789          100   

                description     filename business_unit comparison_type  
0       Values do not match  version.csv   version.csv  Base vs Target  
1       Values do not match  version.csv   version.csv  Base vs Target  
2  Value is missing in base  version.csv   version.csv  Base vs Target  
3  Value is missing in base  version.csv   version.csv  Base vs Target  
4       Values do not match  version.csv   version.csv  Base vs Target  
Target file target\version_1.csv does not exist.


In [29]:
import os
import glob
import pandas as pd
import difflib
import numpy as np
import timeit

def string_diff(str1, str2):
    seq_matcher = difflib.SequenceMatcher(None, str1, str2)
    return seq_matcher.ratio() != 1

def compare_files(base, target, keys):
    chunksize = 10 ** 6  # Define your chunksize here according to your system's memory
    summary_data = []

    for chunk1, chunk2 in zip(pd.read_csv(base, chunksize=chunksize), pd.read_csv(target, chunksize=chunksize)):
        chunk1.set_index(keys, inplace=True)
        chunk2.set_index(keys, inplace=True)

        # Combine columns from both chunks
        columns = set(chunk1.columns).union(set(chunk2.columns))

        for column in columns:
            for row_index in chunk1.index.intersection(chunk2.index):
                value1 = chunk1.at[row_index, column] if column in chunk1.columns and row_index in chunk1.index else np.nan
                value2 = chunk2.at[row_index, column] if column in chunk2.columns and row_index in chunk2.index else np.nan

                description = ''

                if pd.isna(value1) and pd.isna(value2):
                    continue  # Skip if both are NaN
                
                if pd.isna(value1) and not pd.isna(value2):
                    description = 'Value is missing in base'
                elif not pd.isna(value1) and pd.isna(value2):
                    description = 'Value is missing in target'
                elif string_diff(str(value1), str(value2)):
                    description = 'Values do not match'

                if description:
                    keys_dict = dict(zip(keys, row_index))
                    keys_dict.update({
                        'column': column,
                        'value_base': value1,
                        'value_target': value2,
                        'description': description
                    })
                    summary_data.append(keys_dict)

    summary_df = pd.DataFrame(summary_data)
    return summary_df

base_dir = 'base'
target_dir = 'target'
keys = ['key_column1', 'key_column2']  # adjust this to your key columns

base_files = glob.glob(os.path.join(base_dir, '*.csv'))

for base_file in base_files:
    filename = os.path.basename(base_file)
    target_file = os.path.join(target_dir, filename)

    if os.path.isfile(target_file):
        start = timeit.default_timer()

        summary_df = compare_files(base_file, target_file, keys)

        # Add additional columns to the dataframe
        summary_df['filename'] = filename
        summary_df['business_unit'] = 'TH' 
        summary_df['comparison_type'] = 'LBU_Submission_to_Tool_Pack'

        end = timeit.default_timer()
        print('Time: ', end - start, 'seconds')

    else:
        print(f"Target file {target_file} does not exist.")


Time:  0.009159899999758636 seconds
Target file target\version_1.csv does not exist.


In [30]:
summary_df

,key_column1,key_column2,column,value_base,value_target,description,filename,business_unit,comparison_type
0,key2,subkey2,data_column1,World,Earth,Values do not match,version.csv,TH,LBU_Submission_to_Tool_Pack
1,key3,subkey3,data_column1,Example,Sample,Values do not match,version.csv,TH,LBU_Submission_to_Tool_Pack
2,key1,subkey1,data_column3,NaN,A,Value is missing in base,version.csv,TH,LBU_Submission_to_Tool_Pack
3,key3,subkey3,data_column3,NaN,C,Value is missing in base,version.csv,TH,LBU_Submission_to_Tool_Pack
4,key3,subkey3,data_column2,789,100,Values do not match,version.csv,TH,LBU_Submission_to_Tool_Pack
